In [1]:
# Import necessary libraries
import os
import shutil
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import timm

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
num_classes = 10  
model = timm.create_model('mvitv2_large', pretrained=True, num_classes=num_classes)
model.load_state_dict(torch.load('model_epoch_10_augmentations.pth'))
model = model.to(device)
model.eval()

MultiScaleVit(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 144, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
  )
  (stages): ModuleList(
    (0): MultiScaleVitStage(
      (blocks): ModuleList(
        (0): MultiScaleBlock(
          (norm1): LayerNorm((144,), eps=1e-06, elementwise_affine=True)
          (attn): MultiScaleAttention(
            (qkv): Linear(in_features=144, out_features=432, bias=True)
            (proj): Linear(in_features=144, out_features=144, bias=True)
            (pool_q): Conv2d(72, 72, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=72, bias=False)
            (norm_q): LayerNorm((72,), eps=1e-06, elementwise_affine=True)
            (pool_k): Conv2d(72, 72, kernel_size=(3, 3), stride=(4, 4), padding=(1, 1), groups=72, bias=False)
            (norm_k): LayerNorm((72,), eps=1e-06, elementwise_affine=True)
            (pool_v): Conv2d(72, 72, kernel_size=(3, 3), stride=(4, 4), padding=(1, 1), groups=72, bias=False)
            (norm_v): L

In [8]:
transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load validation dataset
val_dataset = datasets.ImageFolder(root='D:\\CVPI\\Dataset\\validation', transform=transform_val)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Initialize lists for predictions and labels
all_preds = []
all_labels = []
misclassified_path = []


In [23]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm  # Import tqdm for progress bar

# Assuming val_dataset is your validation dataset
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)  # Adjust batch size and num_workers as needed

# Initialize lists to store predictions and labels
all_preds = []
all_labels = []

# Evaluation loop with progress tracking
with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(tqdm(val_loader, desc='Evaluating', unit='batch')):
        images, labels = images.to(device), labels.to(device)  # Move to GPU
        logits = model(images)  # Get the logits
        probs = torch.softmax(logits, dim=1)  # Apply softmax to get probabilities

        # Store probabilities and true labels
        all_preds.append(probs)  # Store logits (you can also store raw logits)
        all_labels.append(labels)

# Concatenate all predictions and labels
all_preds = torch.cat(all_preds).cpu().numpy()  # Convert to NumPy array
all_labels = torch.cat(all_labels).cpu().numpy()  # Convert to NumPy array

Evaluating:   0%|                                                                           | 0/505 [00:05<?, ?batch/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 166.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 4.73 GiB is allocated by PyTorch, and 158.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [12]:
import torch
import numpy as np

# Initialize lists to store predictions and labels
all_preds = []
all_labels = []

# Evaluation loop
with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(val_loader):
        images, labels = images.to(device), labels.to(device)
        logits = model(images)  # Get the logits
        probs = torch.softmax(logits, dim=1)  # Apply softmax to get probabilities

        all_preds.extend(probs.cpu().numpy())  # Store probabilities
        all_labels.extend(labels.cpu().numpy())  # Store true labels

# Convert to numpy arrays
all_preds = np.array(all_preds) 
all_labels = np.array(all_labels)


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Example class names
class_names = ['Angioectasia', 'Bleeding', 'Erosion', 'Erythema', 'Foreign Body', 
               'Lymphangiectasia', 'Normal', 'Polyp', 'Ulcer', 'Worms']

# Binarize the output labels
y_bin = label_binarize(all_labels, classes=np.arange(len(class_names)))

# Initialize a figure
plt.figure(figsize=(10, 7))

# Loop through each class to calculate and plot Precision-Recall
for i in range(len(class_names)):
    precision, recall, _ = precision_recall_curve(y_bin[:, i], all_preds[:, i])  # Use probabilities for precision-recall calculation

    # Plotting each class's Precision-Recall curve
    plt.plot(recall, precision, marker='.', label=f'Class {class_names[i]}')

# Adding labels and title
plt.title('Multiclass Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.grid()
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Assuming you have the following class names
class_names = ['Angioectasia', 'Bleeding', 'Erosion', 'Erythema', 'Foreign Body', 
               'Lymphangiectasia', 'Normal', 'Polyp', 'Ulcer', 'Worms']

# Binarize the output labels
y_bin = label_binarize(all_labels, classes=np.arange(len(class_names)))

# Initialize a figure
plt.figure(figsize=(10, 7))

# Loop through each class to calculate and plot ROC curve
for i in range(len(class_names)):
    fpr, tpr, _ = roc_curve(y_bin[:, i], all_preds[:, i])  # Use probabilities for ROC calculation
    roc_auc = auc(fpr, tpr)  # Calculate the area under the ROC curve

    # Plotting each class's ROC curve
    plt.plot(fpr, tpr, label=f'Class {class_names[i]} (area = {roc_auc:.2f})')

# Plot the diagonal line
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
plt.title('Multiclass ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid()
plt.show()


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Calculate precision, recall, f1-score, and support
report = classification_report(all_labels, np.argmax(all_preds, axis=1), output_dict=True)

# Extract metrics for each class
precision = []
recall = []
f1 = []
class_names = []

for class_name in report.keys():
    if class_name not in ['accuracy', 'macro avg', 'weighted avg']:
        precision.append(report[class_name]['precision'])
        recall.append(report[class_name]['recall'])
        f1.append(report[class_name]['f1-score'])
        class_names.append(class_name)

# Convert to numpy arrays for easier manipulation
precision = np.array(precision)
recall = np.array(recall)
f1 = np.array(f1)

# Create a bar graph
x = np.arange(len(class_names))
width = 0.2  # Width of the bars

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width, precision, width, label='Precision')
bars2 = ax.bar(x, recall, width, label='Recall')
bars3 = ax.bar(x + width, f1, width, label='F1 Score')

# Add some text for labels, title, and custom x-axis tick labels, etc.
ax.set_xlabel('Classes')
ax.set_ylabel('Scores')
ax.set_title('Precision, Recall, and F1 Score by Class')
ax.set_xticks(x)
ax.set_xticklabels(class_names)
ax.legend()

plt.show()


In [ ]:
import numpy as np
import torch
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have the following lists: all_labels (true labels) and all_preds (predicted probabilities)
# all_labels: True class labels
# all_preds: Predicted probabilities (from your model)

# Get the predicted classes from probabilities
predicted_classes = np.argmax(all_preds, axis=1)

# Generate the classification report
report = classification_report(all_labels, predicted_classes, output_dict=True)

# Extract precision, recall, and F1 scores
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f1_score = report['macro avg']['f1-score']

print("Macro Average Precision:", precision)
print("Macro Average Recall:", recall)
print("Macro Average F1 Score:", f1_score)

# Print confusion matrix
conf_matrix = confusion_matrix(all_labels, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.arange(10), yticklabels=np.arange(10))
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
import numpy as np
import torch
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have the following lists: all_labels (true labels) and all_preds (predicted probabilities)
# all_labels: True class labels
# all_preds: Predicted probabilities (from your model)

# Get the predicted classes from probabilities
predicted_classes = np.argmax(all_preds, axis=1)

# Generate the classification report
report = classification_report(all_labels, predicted_classes, output_dict=True)

# Extract precision, recall, and F1 scores for each class
for class_name, metrics in report.items():
    if class_name not in ['accuracy', 'macro avg', 'weighted avg']:
        precision = metrics['precision']
        recall = metrics['recall']
        f1_score = metrics['f1-score']
        print(f"Class: {class_name} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")

# Print confusion matrix
conf_matrix = confusion_matrix(all_labels, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.arange(10), yticklabels=np.arange(10))
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()
